In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=cf8a3649210d15abb66a7ddde608ffebaf88f67c94661b8cfd632b3b1e196f6f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubu

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
# https://drive.google.com/file/d/1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL/view?usp=drive_link

id='1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystoreuserreviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df1 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
df2 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystoreuserreviews.csv', header=True, inferSchema=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Desde los ejercicios de pandas sabemos que hay una linea con Rating 19, lo cual no es posible, la sacamos

In [ ]:
rddreviews = df2.rdd
rddapps = df1.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame").cache()

Vemos un poco como se presentan los datos de cada registro

In [ ]:
rddapps.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

In [ ]:
rddreviews.take(5)

[Row(App='10 Best Foods for You', Translated_Review='I like eat delicious food. That\'s I\'m cooking food myself, case "10 Best Foods" helps lot, also "Best Before (Shelf Life)"', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.5333333333333333'),
 Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

Vemos que hay apps repetidas

In [ ]:
aux = rddapps.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
aux1 = aux.filter(lambda x: x[1] > 1)
aux1.count()

798

En mi criterio las sacamos

In [ ]:
rddapps1 = rddapps.map(lambda x: (x[0], x)).reduceByKey(lambda x, y: x)
rddapps1.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook',
  Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')),
 ('Coloring book moana',
  Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up')),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver=

Y nos quedamos unicamente con los juegos de accion

In [ ]:
rddapps2 = rddapps1.filter(lambda x: x[1].Category == "GAME" and "ACTION" in x[1][9].upper()).map(lambda x: x[1])
rddapps2.take(5)

[Row(App='ROBLOX', Category='GAME', Rating=4.5, Reviews='4447388', Size='67M', Installs='100,000,000+', Type='Free', Price='0', Content Rating='Everyone 10+', Genres='Adventure;Action & Adventure', Last Updated='July 31, 2018', Current Ver='2.347.225742', Android Ver='4.1 and up'),
 Row(App='slither.io', Category='GAME', Rating=4.4, Reviews='5234162', Size='Varies with device', Installs='100,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Action', Last Updated='November 14, 2017', Current Ver='Varies with device', Android Ver='2.3 and up'),
 Row(App='Temple Run 2', Category='GAME', Rating=4.3, Reviews='8118609', Size='62M', Installs='500,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Action', Last Updated='July 5, 2018', Current Ver='1.49.1', Android Ver='4.0 and up'),
 Row(App='Helix Jump', Category='GAME', Rating=4.2, Reviews='1497361', Size='33M', Installs='100,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Action',

Vemos que al hacer esto hay 5 generos distintos con la palabra ACTION

In [ ]:
rdd_generos = rddapps2.map(lambda x: x[9]).cache()
rdd_generos.distinct().count()

5

Hacemos un collect ya que sabemos que son pocos, y vemos que ademas de los generos accion tambien nos quedamos con las apps de accion y aventura
los desechamos (salvo que tambien el juego sea de accion unicamente)

In [ ]:
rdd_generos.distinct().collect()

['Adventure;Action & Adventure',
 'Action',
 'Casual;Action & Adventure',
 'Action;Action & Adventure',
 'Simulation;Action & Adventure']

Comprobamos que solo nos quedamos con aquellas apps de accion (podemos hacer collect ya que sabemos que van a ser menos generos que los
que habian antes de filtrar)

In [ ]:
rddapps3 = rddapps2.filter(lambda x: (x.Genres == "Action" or x.Genres =="Action;Action & Adventure"))
rddapps3.map(lambda x: x[9]).distinct().collect()

['Action', 'Action;Action & Adventure']

Nos quedamos con el nombre de la app para luego unirlo con las reviews

In [ ]:
rddapps4 = rddapps3.map(lambda x: (x[0], x[1], x[9]))
rddapps4.take(5)

[('slither.io', 'GAME', 'Action'),
 ('Temple Run 2', 'GAME', 'Action'),
 ('Helix Jump', 'GAME', 'Action'),
 ('Zombie Hunter King', 'GAME', 'Action'),
 ('Kick the Buddy', 'GAME', 'Action')]

Del rdd de reviews nos quedamos unicamente con aquellas reviews positivas

In [ ]:
rddreviews1 = rddreviews.filter(lambda x: x.Sentiment == "Positive").cache()
rddreviews1.count()

23998

Unimos los rdd de reviews asi nos queda cada app con su nombre y una reseña

In [ ]:
rddcombinado = rddapps4.join(rddreviews1)
rddcombinado.take(5)

[('DRAGON BALL LEGENDS',
  ('GAME',
   'The ultimate Dragon Ball mobile game every fan waiting for! With real time action, reaction based controls easy learn hard master, best action mobile games massive franchise. Also last least, real time PvP real players around world!! Extremely fun bright future ahead. With many characters throughout series, game lucky replace competitors definitive Dragon Ball mobile game!')),
 ('DRAGON BALL LEGENDS',
  ('GAME',
   "The game mad success. So many things done right mobile game. There's lot characters choose from, draw. The random draws definitely weighed nicely occasionally get really good characters. The fighting actually feels satisfying! You dodge, special attacks, even play online. It's nice change pace card games generic stuff always releases. This definitely puzzle bobble game. 5 Stars, easily.")),
 ('DRAGON BALL LEGENDS',
  ('GAME',
   'I feel compelled leave review. By far best dragon ball game. You love absolutely everything it. The game p

Sumamos la cantidad de reviews positivas de cada app y nos quedamos con una reseña de cada una

In [ ]:
def funcion(a, b):
    count = a[0] + b[0]
    random_review = a[1]
    return (count, random_review)

rddcombinado1 = rddcombinado.map(lambda x: (x[0], (1, x[1][1])))
rddcombinado2 = rddcombinado1.reduceByKey(funcion)
rddcombinado2.take(5)

[('DRAGON BALL LEGENDS',
  (127,
   'The ultimate Dragon Ball mobile game every fan waiting for! With real time action, reaction based controls easy learn hard master, best action mobile games massive franchise. Also last least, real time PvP real players around world!! Extremely fun bright future ahead. With many characters throughout series, game lucky replace competitors definitive Dragon Ball mobile game!')),
 ('Crossy Road',
  (67,
   'Hi there, game great, even better cool characters like disco zoo, flying disco ball, dancing animals weird music funny, I idea could help game, favorite list works u add chatacter favorite u chatacters fa... Full Review')),
 ('Anger of stick 5 : zombie', (2, 'Keep make game better.')),
 ('CATS: Crash Arena Turbo Stars',
  (35,
   'After playing game year fun. The issue I noticing trend gang fights. They seem set make pay actually get 100. Also, get gems like used to. Shows thwy care money fun. I know e... Full Review')),
 ('Dragon Hills',
  (10,
   

Por ultimo, devolvemos aquellos 3 juegos de accion con mas reviews positivas y una reseña de cada uno

In [ ]:
def takeSecond(elem):
    return elem[1]*-1

def threeMax(apps):
  apps.sort(key=takeSecond)
  return apps[:3]

rddcombinado2.map(lambda x: ((x[0],x[1][0], x[1][1]), ('',0,''))).reduce(lambda x,y: threeMax([x[0],x[1],y[0],y[1]]))

[('Helix Jump',
  209,
  "Update: I finally bought premium version avoid ads I'm still getting ads non stop despite reaching devs responded all. Going request refund I guess. Original review: The game fun addicting, many ads. An ad every time restart level, ad beat level, ad blink many times. I know devs make money game free pretty top"),
 ('Bowmasters',
  169,
  "Really like game far. Great gameplay lots fun. Edit: I took away star constant asking rate even I've already rated taking away chances chests. Just seems excessive. It's good game, people need badgered rating."),
 ('Garena Free Fire',
  136,
  'Please add bolt action Sniper rifles. That would great. The game way easy. And also please decrease amount vehicles per game? Or reduce vehicles except monster truck (because airdrop vehicle). The game way easy I think. Just increase recoil little players say made hard. Nice game. Keep Garena.')]